In [1]:
%run -n main.py
dotenv = dict(read_dotenv('.env'))
openai.api_key = dotenv['OPENAI_TOKEN']

# sources

## alpaca

In [ ]:
!mkdir -p data/sources/alpaca
!curl -L https://github.com/yizhongw/self-instruct/raw/main/human_eval/user_oriented_instructions.jsonl \
    > data/sources/alpaca/user_oriented_instructions.jsonl

In [ ]:
%run -n main.py
items = read_jsonl('data/sources/alpaca/user_oriented_instructions.jsonl')
alpaca_items = list(parse_alpaca(items))

## vicuna

In [ ]:
!mkdir -p data/sources/vicuna
!curl -L https://github.com/lm-sys/vicuna-blog-eval/raw/main/eval/table/question.jsonl \
    > data/sources/vicuna/question.jsonl

In [ ]:
%run -n main.py
items = read_jsonl('data/sources/vicuna/question.jsonl')
vicuna_items = list(parse_vicuna(items))

## arena

In [ ]:
!mkdir -p data/sources/arena
!curl -L curl -L https://huggingface.co/datasets/lmsys/chatbot_arena_conversations/resolve/main/data/train-00000-of-00001-cced8514c7ed782a.parquet \
    > data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet

In [ ]:
%run -n main.py
records = pd.read_parquet('data/sources/arena/train-00000-of-00001-cced8514c7ed782a.parquet').itertuples()
arena_items = list(parse_arena(records))

# orig

In [ ]:
orig_items = alpaca_items + vicuna_items

In [ ]:
instruction_items = {
    _['instruction']: _
    for _ in arena_items
    if _['lang'] == 'English'
}
orig_items.extend(random.sample(list(instruction_items.values()), 1000))

In [ ]:
write_jsonl('data/orig.jsonl', orig_items)

In [12]:
orig_items = list(read_jsonl('data/orig.jsonl'))
random.sample(orig_items, 5)

[{'id': '6711af7d-4165-45e2-a45e-755c4eca5026',
  'source': 'arena',
  'source_id': '1ad24688c2a545e4b40be8c8f0129a8f',
  'lang': 'English',
  'instruction': 'convert movie to emoji : midnight express, shinning'},
 {'id': '418214b0-fdb9-4634-ad50-0701b1bab193',
  'source': 'arena',
  'source_id': 'ca5b0426f2ab48f6b0a82ddeace44480',
  'lang': 'English',
  'instruction': 'can you create me 3 music band name with that acronym: DIK and give a description'},
 {'id': '4d5561ff-d153-4f84-8def-8d15525e0013',
  'source': 'alpaca',
  'source_id': 'user_oriented_task_67',
  'instruction': 'You should choose a YouTube video title based on the video\'s content. A video\'s title tells viewers what to expect from it. It should be direct, honest, and clear. The title of the video needs to capture the attention of viewers, so do not use an unclear or ambiguous one.\n\n"A research study has been conducted to determine if exercise really can "boost" your metabolism."'},
 {'id': 'e96d61ad-7d3a-4fbf-b2b4-6

# translate

In [ ]:
translate_items = [
    {
        'id': _['id'],
        'instruction': _['instruction'],
        'answer': None
    }
    for _ in orig_items
]

In [ ]:
%run -n main.py
items = [_ for _ in translate_items if not _['answer']]
queue = iter(tqdm(items))
workers = [openai_translate_worker(queue) for _ in range(10)]
await asyncio.gather(*workers);

In [ ]:
write_jsonl('data/translate.jsonl', translate_items)

# label studio

In [ ]:
%run -n main.py
label_studio = label_studio_sdk.Client('http://localhost:8080', dotenv['LABELSTUDIO_TOKEN'])
label_studio.check_connection()

In [ ]:
title_projects = {
    _.title: _
    for _ in label_studio.list_projects()
}
translate_project = title_projects['translate']
classify_project = title_projects['classify']

# translate annot

In [ ]:
translate_items = read_jsonl('data/translate.jsonl')
annot_items = [translate_annot_item(_) for _ in translate_items]
random.choice(annot_items)

In [ ]:
translate_project.delete_all_tasks();
translate_project.import_tasks(annot_items);

In [ ]:
annot_items = translate_project.export_tasks()
translate_items = [annot_translate_item(_) for _ in annot_items]
random.sample(translate_items, 3)

In [ ]:
write_jsonl('data/translate.jsonl', translate_items)

# classify

In [ ]:
%run -n main.py
classify_items = list(read_jsonl('data/classify.jsonl'))
id_embeddings = read_pickle('data/embeddings.pkl')

In [ ]:
%run -n main.py
items = [
    _ for _ in classify_items
    if _['id'] not in id_embeddings
]
for index in tqdm(range(0, len(items), 64)):
    batch = items[index:index + 64]
    texts = [_['instruction'] for _ in batch]
    embeddings = openai_embed_batch(texts)
    for item, embedding in zip(batch, embeddings):
        id_embeddings[item['id']] = np.array(embedding)
write_pickle('data/embeddings.pkl', id_embeddings)

In [ ]:
target_items = [
    _ for _ in classify_items
    if _['tags'] and 'bad instruction' not in _['tags']
]
items = [_ for _ in classify_items if not _['tags']]

for item in tqdm(items):
    max_sim = 0
    for target_item in target_items:
        sim = cosine_sim(
            id_embeddings[item['id']],
            id_embeddings[target_item['id']]
        )
        if sim > max_sim:
            max_sim = sim
            item['tags'] = target_item['tags']
    item['max_sim'] = max_sim

In [ ]:
items = []
for item in classify_items:
    if not item.get('max_sim'):
        continue
        
    if 'enumerate' not in item['tags']:
        continue
        
    items.append(item)

items = sorted(items, key=lambda _: _['max_sim'], reverse=False)
annot_items = [classify_annot_item(_) for _ in items]
len(annot_items)

In [ ]:
classify_project.delete_all_tasks();
classify_project.import_tasks(annot_items);

In [ ]:
%run -n main.py
annot_items = classify_project.export_tasks()
items = (annot_classify_item(_) for _ in annot_items)
id_tags = {
    _['id']: _['tags']
    for _ in items
}
for item in classify_items:
    tags = id_tags.get(item['id'])
    if tags is not None:
        item['tags'] = tags
        item.pop('max_sim', None)

In [ ]:
for item in classify_items:
    if item.pop('max_sim', None):
        item['tags'] = []
write_jsonl('data/classify.jsonl', classify_items)

# tasks

In [13]:
items = read_jsonl('data/classify.jsonl')
id_tags = {_['id']: _['tags'] for _ in items if _['tags']}
len(id_tags)

551

In [14]:
items = read_jsonl('data/orig.jsonl')
id_sources = {_['id']: _['source'] for _ in items}
len(id_sources)

1317

In [16]:
task_items = []
items = read_jsonl(f'data/translate.jsonl')
for item in items:
    id = item['id']
    tags = id_tags.get(id, [])
    if 'bad instruction' in tags:
        continue

    task_items.append({
        'id': id,
        'source': id_sources[id],
        'instruction': item['answer'],
        'tags': tags
    })
write_jsonl(f'data/tasks.jsonl', task_items)

In [21]:
task_items = list(read_jsonl('data/tasks.jsonl'))

In [27]:
%run -n main.py
source_items = defaultdict(list)
for item in task_items:
    source_items[item['source']].append(item)

with open('data/README.md', 'w') as file:
    with redirect_stdout(file):
        for source in ['alpaca', 'vicuna', 'arena']:
            print(f'<h1>{source}</h1>')

            items = [_ for _ in source_items[source] if _['tags']]
            for item in random.sample(items, 30):
                print(' '.join(f'<code>#{_}</code>' for _ in item['tags']))
                print('<br/>')
                instruction = html.escape(item['instruction'])
                print('<br/>\n'.join(instruction.splitlines()))
                print('<br/><br/>')

# infer